In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from tqdm import tqdm

# Load data

#### Trips

In [2]:
trips = pd.read_csv('../detroit/output/final_trips.csv', index_col='trip_id')

/home/ubuntu/anaconda3/envs/baus/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
trips["inbound"] = ~trips.outbound

#### Joint Tour Participants

In [30]:
jtp = pd.read_csv('../detroit/output/final_joint_tour_participants.csv', index_col=0)

#### Tours

In [5]:
tours = pd.read_csv('../detroit/output/final_tours.csv', index_col='tour_id')

/home/ubuntu/anaconda3/envs/baus/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (16,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
trips["tour_start"] = trips.tour_id.map(tours.start)

In [7]:
trips["tour_end"] = trips.tour_id.map(tours.end)

#### BEAM Plans

In [5]:
plans = pd.read_csv('../detroit/output/plans.csv', index_col=0)

#### Persons

In [8]:
persons = pd.read_csv('../detroit/output/final_persons.csv', index_col=0)

# Exploration

#### Topologically sorted?

In [9]:
topo_sort_mask = (
        (trips['destination'].shift() == trips['origin']) |
        (trips['person_id'].shift() != trips['person_id']))

In [10]:
topo_sort_mask.value_counts()

True     13786830
False      408037
dtype: int64

#### Chronologically sorted?

In [11]:
chrono_sort_mask = (
        (trips['depart'].shift() <= trips['depart']) |
        (trips['person_id'].shift() != trips['person_id']))

In [12]:
chrono_sort_mask.value_counts()

True     13248998
False      945869
dtype: int64

#### Order as best as possible

In [109]:
initial_sort_cols = ['person_id', 'depart', 'tour_start', 'tour_end', 'tour_id', 'inbound', 'trip_num']

In [38]:
ordered_trips = trips.sort_values(initial_sort_cols)

In [64]:
topo_sort_mask = (
        (ordered_trips['destination'].shift() == ordered_trips['origin']) |
        (ordered_trips['person_id'].shift() != ordered_trips['person_id']))

In [65]:
topo_sort_mask.value_counts()

True     14108751
False       86116
dtype: int64

In [21]:
len(ordered_trips[~topo_sort_mask]['person_id'].unique())

44000

In [67]:
ordered_trips['topo_sorted'] = topo_sort_mask

In [68]:
unsorted_persons = ordered_trips[~topo_sort_mask]['person_id'].unique()

In [69]:
ordered_trips[ordered_trips['person_id'] == np.random.choice(unsorted_persons)]

,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode,inbound,tour_start,tour_end,topo_sorted
trip_id,,,,,,,,,,,,,,,,
199039289,606827,234482,24879911,atwork,1,True,1,atwork,425,947,8.0,SHARED2PAY,False,8.0,10.0,True
199039569,606827,234482,24879946,work,1,True,1,work,947,108,8.0,WALK_LOC,False,8.0,17.0,False
199039293,606827,234482,24879911,atwork,1,False,1,Work,947,425,10.0,SHARED2FREE,True,8.0,10.0,False
199039573,606827,234482,24879946,work,1,False,1,Home,108,947,17.0,WALK_LOC,True,8.0,17.0,True


In [76]:
ordered_trips[ordered_trips['person_id'] == 3886426]

,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode,inbound,tour_start,tour_end,topo_sorted
trip_id,,,,,,,,,,,,,,,,
1274747993,3886426,1607701,159343499,shopping,1,True,1,shopping,295,153,8.0,SHARED2PAY,False,8.0,11.0,True
1274747997,3886426,1607701,159343499,shopping,1,False,1,Home,153,295,11.0,SHARED2PAY,True,8.0,11.0,True
1274747953,3886426,1607701,159343494,othmaint,1,True,3,othmaint,402,153,11.0,WALK_LOC,False,11.0,14.0,True
1274747957,3886426,1607701,159343494,othmaint,1,False,2,shopping,565,402,12.0,WALK_LOC,True,11.0,14.0,True
1274747954,3886426,1607701,159343494,othmaint,2,True,3,eatout,402,402,13.0,WALK,False,11.0,14.0,False
1274747955,3886426,1607701,159343494,othmaint,3,True,3,othmaint,402,402,13.0,WALK_LOC,False,11.0,14.0,True
1274747958,3886426,1607701,159343494,othmaint,2,False,2,Home,153,565,14.0,WALK_LOC,True,11.0,14.0,False
1274748001,3886426,1607701,159343500,shopping,1,True,1,shopping,103,153,14.0,SHARED2FREE,False,14.0,18.0,True
1274748005,3886426,1607701,159343500,shopping,1,False,1,Home,153,103,18.0,SHARED2PAY,True,14.0,18.0,True


In [63]:
ordered_trips[ordered_trips['person_id'] == 3886426].sort_values(['tour_start', 'inbound', 'trip_num'])[[
    'person_id', 'tour_id', 'outbound', 'trip_num', 'primary_purpose', 'purpose', 'origin', 'destination', 'depart', 'tour_start', 'tour_end']]

,person_id,tour_id,outbound,trip_num,primary_purpose,purpose,origin,destination,depart,tour_start,tour_end
trip_id,,,,,,,,,,,
1274747993,3886426,159343499,True,1,shopping,shopping,153,295,8.0,8.0,11.0
1274747997,3886426,159343499,False,1,shopping,Home,295,153,11.0,8.0,11.0
1274747953,3886426,159343494,True,1,othmaint,othmaint,153,402,11.0,11.0,14.0
1274747954,3886426,159343494,True,2,othmaint,eatout,402,402,13.0,11.0,14.0
1274747955,3886426,159343494,True,3,othmaint,othmaint,402,402,13.0,11.0,14.0
1274747957,3886426,159343494,False,1,othmaint,shopping,402,565,12.0,11.0,14.0
1274747958,3886426,159343494,False,2,othmaint,Home,565,153,14.0,11.0,14.0
1274748001,3886426,159343500,True,1,shopping,shopping,153,103,14.0,14.0,18.0
1274748005,3886426,159343500,False,1,shopping,Home,103,153,18.0,14.0,18.0


In [49]:
ordered_trips[ordered_trips['person_id'] == 1995735].sort_values(['tour_start', 'inbound', 'trip_num'])

,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode,inbound,tour_start,tour_end
trip_id,,,,,,,,,,,,,,,
654601113,1995735,815905,81825139,atwork,1,True,1,atwork,52,171,8.0,SHARED2PAY,False,8.0,8.0
654601393,1995735,815905,81825174,work,1,True,1,work,171,558,8.0,SHARED2PAY,False,8.0,20.0
654601117,1995735,815905,81825139,atwork,1,False,1,Work,171,52,8.0,SHARED2FREE,True,8.0,8.0
654601397,1995735,815905,81825174,work,1,False,2,work,607,171,18.0,SHARED2FREE,True,8.0,20.0
654601398,1995735,815905,81825174,work,2,False,2,Home,558,607,20.0,WALK,True,8.0,20.0


In [62]:
ordered_trips[ordered_trips['person_id'] == 3499960].sort_values(['tour_start', 'inbound', 'trip_num'])[[
    'person_id', 'tour_id', 'outbound', 'trip_num', 'primary_purpose', 'purpose', 'origin', 'destination', 'depart', 'tour_start', 'tour_end']]

,person_id,tour_id,outbound,trip_num,primary_purpose,purpose,origin,destination,depart,tour_start,tour_end
trip_id,,,,,,,,,,,
1147986953,3499960,143498369,True,1,escort,escort,112,34,11.0,11.0,12.0
1147986954,3499960,143498369,True,2,escort,escort,34,359,12.0,11.0,12.0
1147986957,3499960,143498369,False,1,escort,escort,359,293,11.0,11.0,12.0
1147986958,3499960,143498369,False,2,escort,Home,293,112,12.0,11.0,12.0
1147986961,3499960,143498370,True,1,escort,escort,112,183,13.0,13.0,14.0
1147986965,3499960,143498370,False,1,escort,Home,183,112,14.0,13.0,14.0


In [170]:
unordered_trips = ordered_trips.loc[ordered_trips.person_id.isin(unsorted_persons), :]

In [150]:
unordered_trips.person_id.unique().shape

(44000,)

In [151]:
reordered = []

In [194]:
choices = unordered_trips.loc[unordered_trips['outbound'] == True, 'depart']

In [198]:
unordered_trips['outbound_departure'] = choices.reindex(unordered_trips.index)

/home/ubuntu/anaconda3/envs/baus/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [216]:
max_outbound_person_departures = unordered_trips.groupby(['person_id', 'tour_id'])['outbound_departure'].max()

In [222]:
max_outbound_person_departures

person_id  tour_id  
35         1460         16.0
           1463         13.0
           1468         15.0
141        5785         13.0
           5820          8.0
                        ... 
4393280    180124519     5.0
4393314    180125899     8.0
           180125907    18.0
4393481    180132744    18.0
           180132760     8.0
Name: outbound_departure, Length: 102522, dtype: float64

In [225]:
%%time
unordered_trips['max_outbound_departure'] = list(zip(unordered_trips['person_id'], unordered_trips['tour_id']))
unordered_trips['max_outbound_departure'] = unordered_trips['max_outbound_departure'].map(max_outbound_person_departures)

/home/ubuntu/anaconda3/envs/baus/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


CPU times: user 824 ms, sys: 32.1 ms, total: 857 ms
Wall time: 853 ms


/home/ubuntu/anaconda3/envs/baus/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [235]:
unordered_trips['earliest'] = unordered_trips['tour_start']

/home/ubuntu/anaconda3/envs/baus/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [236]:
unordered_trips['earliest'] = unordered_trips[['earliest', 'max_outbound_departure']].max(axis=1)

/home/ubuntu/anaconda3/envs/baus/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [238]:
unordered_trips[unordered_trips['max_outbound_departure'] != unordered_trips['earliest']]

,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,...,trip_mode,inbound,tour_start,tour_end,topo_sorted,atwork_inbound_sorting,atwork_outbound_sorting,outbound_departure,max_outbound_departure,earliest
trip_id,,,,,,,,,,,,,,,,,,,,,


In [218]:
gene_d = {('chrom1', '-', 1234) : 'geneA', ('chrom1', '+', 5678): 'geneB', 
    ('chrom1', '+', 9876): 'geneC', ('chrom2', '+', 13579): 'geneD',
    ('chrom2', '+', 8497): 'geneE', ('chrom2', '-', 98765): 'geneF', 
    ('chrom2', '-', 76856): 'geneG'}

In [219]:
gene_d

{('chrom1', '-', 1234): 'geneA',
 ('chrom1', '+', 5678): 'geneB',
 ('chrom1', '+', 9876): 'geneC',
 ('chrom2', '+', 13579): 'geneD',
 ('chrom2', '+', 8497): 'geneE',
 ('chrom2', '-', 98765): 'geneF',
 ('chrom2', '-', 76856): 'geneG'}

In [214]:
%%time
unordered_trips['max_outbound_departure'] = unordered_trips.person_id.map(max_outbound_person_departures)

CPU times: user 6.03 ms, sys: 204 µs, total: 6.23 ms
Wall time: 5.08 ms


/home/ubuntu/anaconda3/envs/baus/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [215]:
unordered_trips

,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode,inbound,tour_start,tour_end,topo_sorted,atwork_inbound_sorting,atwork_outbound_sorting,outbound_departure,max_outbound_departure
trip_id,,,,,,,,,,,,,,,,,,,,
11705,35,8,1463,othmaint,1,True,2,othmaint,246,847,11.0,SHARED3FREE,False,11.0,13.0,True,2,1,11.0,16.0
11709,35,8,1463,othmaint,1,False,2,shopping,907,903,12.0,SHARED3PAY,True,11.0,13.0,False,2,1,NaN,16.0
11706,35,8,1463,othmaint,2,True,2,othmaint,903,246,13.0,SHARED3FREE,False,11.0,13.0,False,2,1,13.0,16.0
11710,35,8,1463,othmaint,2,False,2,Home,847,907,13.0,SHARED3FREE,True,11.0,13.0,False,2,1,NaN,16.0
11745,35,8,1468,shopping,1,True,1,shopping,907,847,15.0,SHARED2PAY,False,15.0,16.0,True,2,1,15.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441062085,4393481,1812795,180132760,work,1,False,1,Home,132,339,18.0,WALK_LOC,True,8.0,18.0,True,2,1,NaN,18.0
1441061953,4393481,1812795,180132744,atwork,1,True,1,atwork,535,339,18.0,WALK,False,18.0,18.0,False,1,2,18.0,18.0
1441061957,4393481,1812795,180132744,atwork,1,False,3,work,339,535,18.0,WALK,True,18.0,18.0,True,1,2,NaN,18.0


In [212]:
%%time
unordered_trips.join(max_outbound_person_departures, on='person_id')

CPU times: user 26.8 ms, sys: 8.33 ms, total: 35.1 ms
Wall time: 34.3 ms


,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode,inbound,tour_start,tour_end,topo_sorted,atwork_inbound_sorting,atwork_outbound_sorting,outbound_departure,max_outbound_departure
trip_id,,,,,,,,,,,,,,,,,,,,
11705,35,8,1463,othmaint,1,True,2,othmaint,246,847,11.0,SHARED3FREE,False,11.0,13.0,True,2,1,11.0,16.0
11709,35,8,1463,othmaint,1,False,2,shopping,907,903,12.0,SHARED3PAY,True,11.0,13.0,False,2,1,NaN,16.0
11706,35,8,1463,othmaint,2,True,2,othmaint,903,246,13.0,SHARED3FREE,False,11.0,13.0,False,2,1,13.0,16.0
11710,35,8,1463,othmaint,2,False,2,Home,847,907,13.0,SHARED3FREE,True,11.0,13.0,False,2,1,NaN,16.0
11745,35,8,1468,shopping,1,True,1,shopping,907,847,15.0,SHARED2PAY,False,15.0,16.0,True,2,1,15.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441062085,4393481,1812795,180132760,work,1,False,1,Home,132,339,18.0,WALK_LOC,True,8.0,18.0,True,2,1,NaN,18.0
1441061953,4393481,1812795,180132744,atwork,1,True,1,atwork,535,339,18.0,WALK,False,18.0,18.0,False,1,2,18.0,18.0
1441061957,4393481,1812795,180132744,atwork,1,False,3,work,339,535,18.0,WALK,True,18.0,18.0,True,1,2,NaN,18.0


In [171]:
primary_purposes = trips['primary_purpose'].unique()
atwork_sorting_inbound = {purpose: 1 if purpose == 'atwork' else 2 for purpose in primary_purposes}
atwork_sorting_outbound = {purpose: 2 if purpose == 'atwork' else 1 for purpose in primary_purposes}

In [172]:
unordered_trips['atwork_inbound_sorting'] = unordered_trips['primary_purpose'].map(atwork_sorting_inbound).values
unordered_trips['atwork_outbound_sorting'] = unordered_trips['primary_purpose'].map(atwork_sorting_outbound).values

/home/ubuntu/anaconda3/envs/baus/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/ubuntu/anaconda3/envs/baus/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [187]:
reordered = []
processed_persons = []

In [188]:
for person_id, person_trips in unordered_trips.groupby('person_id'):
    
    if person_id in processed_persons:
        continue
    
    # unsorted trips due to misplaced atwork tour:
    # trips end with an atwork tour
    if person_trips['primary_purpose'].iloc[-1] == 'atwork':
        reordered_person_trips = person_trips.sort_values(['depart', 'inbound', 'atwork_inbound_sorting', 'trip_num'])
        if all(reordered_person_trips['origin'][1:].values == reordered_person_trips['destination'][:-1].values):
            reordered.append(reordered_person_trips)
            processed_persons.append(person_id)
        else:
            print('atwork inbound sorting failed!')
            break

    # trips start with an atwork tour
    elif person_trips['primary_purpose'].iloc[0] == 'atwork':
        reordered_person_trips = person_trips.sort_values(['depart', 'inbound', 'atwork_outbound_sorting', 'trip_num'])
        if all(reordered_person_trips['origin'][1:].values == reordered_person_trips['destination'][:-1].values):
            reordered.append(reordered_person_trips)
            processed_persons.append(person_id)
        else:
            print('atwork outbound sorting failed!')
            break
    
    else:
        
        # if unsorted trips due to first inbound occuring before last outbound
        first_unsorted = person_trips.loc[person_trips['topo_sorted'] == False].iloc[0]
        if first_unsorted[['outbound', 'trip_num']].tolist() == [False, 1]:
            reordered_person_trips = person_trips.sort_values(['tour_start', 'inbound', 'trip_num'])
            if all(reordered_person_trips['origin'][1:].values == reordered_person_trips['destination'][:-1].values):
                reordered.append(reordered_person_trips)
                processed_persons.append(person_id)
            else:
                print('Oops!')
                break
    
        # else unsorted due to sorting by shared departure hours
        else:

            if person_trips['purpose'].iloc[-1] != 'Home':
                print('Last trip not home!')
                break
            else:
                print('Hmmm...')
                break

Oops!


In [189]:
person_trips

,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode,inbound,tour_start,tour_end,topo_sorted,atwork_inbound_sorting,atwork_outbound_sorting
trip_id,,,,,,,,,,,,,,,,,,
249265,759,210,31158,work,1,True,1,work,852,821,8.0,SHARED2FREE,False,8.0,17.0,True,2,1
248953,759,210,31119,atwork,1,True,1,atwork,394,852,11.0,SHARED2PAY,False,11.0,14.0,True,1,2
249269,759,210,31158,work,1,False,2,escort,336,852,14.0,SHARED2PAY,True,8.0,17.0,False,2,1
248957,759,210,31119,atwork,1,False,1,Work,852,394,14.0,SHARED2FREE,True,11.0,14.0,False,1,2
249270,759,210,31158,work,2,False,2,Home,821,336,17.0,DRIVEALONEPAY,True,8.0,17.0,False,2,1


In [190]:
reordered_person_trips

,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode,inbound,tour_start,tour_end,topo_sorted,atwork_inbound_sorting,atwork_outbound_sorting
trip_id,,,,,,,,,,,,,,,,,,
249265,759,210,31158,work,1,True,1,work,852,821,8.0,SHARED2FREE,False,8.0,17.0,True,2,1
249269,759,210,31158,work,1,False,2,escort,336,852,14.0,SHARED2PAY,True,8.0,17.0,False,2,1
249270,759,210,31158,work,2,False,2,Home,821,336,17.0,DRIVEALONEPAY,True,8.0,17.0,False,2,1
248953,759,210,31119,atwork,1,True,1,atwork,394,852,11.0,SHARED2PAY,False,11.0,14.0,True,1,2
248957,759,210,31119,atwork,1,False,1,Work,852,394,14.0,SHARED2FREE,True,11.0,14.0,False,1,2


In [111]:
initial_sort_cols

['person_id',
 'depart',
 'tour_start',
 'tour_end',
 'tour_id',
 'inbound',
 'trip_num']

In [133]:
person_trips['primary_purpose'].iloc[-1] == 'atwork':

'atwork'

In [176]:
person_trips.sort_values(['depart', 'inbound', 'trip_num', 'atwork_inbound_sorting'])

,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode,inbound,tour_start,tour_end,topo_sorted,atwork_inbound_sorting,atwork_outbound_sorting
trip_id,,,,,,,,,,,,,,,,,,
46561,141,41,5820,work,1,True,2,escort,907,847,6.0,SHARED3PAY,False,6.0,13.0,True,2,1
46562,141,41,5820,work,2,True,2,work,486,907,8.0,DRIVEALONEFREE,False,6.0,13.0,True,2,1
46281,141,41,5785,atwork,1,True,1,atwork,33,486,13.0,SHARED2PAY,False,13.0,13.0,False,1,2
46285,141,41,5785,atwork,1,False,1,Work,486,33,13.0,SHARED2PAY,True,13.0,13.0,True,1,2
46565,141,41,5820,work,1,False,2,othmaint,486,486,13.0,DRIVEALONEFREE,True,6.0,13.0,True,2,1
46566,141,41,5820,work,2,False,2,Home,847,486,13.0,DRIVEALONEFREE,True,6.0,13.0,True,2,1


In [177]:
reordered_person_trips = person_trips.sort_values(['depart', 'inbound', 'trip_num', 'atwork_inbound_sorting'])

In [178]:
all(reordered_person_trips['origin'][1:].values == reordered_person_trips['destination'][:-1].values)

True

In [146]:
person_trips.sort_values(['depart', 'inbound', 'trip_num', 'atwork_inbound_sorting'])

,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode,inbound,tour_start,tour_end,topo_sorted,atwork_inbound_sorting
trip_id,,,,,,,,,,,,,,,,,
46561,141,41,5820,work,1,True,2,escort,907,847,6.0,SHARED3PAY,False,6.0,13.0,True,2
46562,141,41,5820,work,2,True,2,work,486,907,8.0,DRIVEALONEFREE,False,6.0,13.0,True,2
46281,141,41,5785,atwork,1,True,1,atwork,33,486,13.0,SHARED2PAY,False,13.0,13.0,False,1
46285,141,41,5785,atwork,1,False,1,Work,486,33,13.0,SHARED2PAY,True,13.0,13.0,True,1
46565,141,41,5820,work,1,False,2,othmaint,486,486,13.0,DRIVEALONEFREE,True,6.0,13.0,True,2
46566,141,41,5820,work,2,False,2,Home,847,486,13.0,DRIVEALONEFREE,True,6.0,13.0,True,2


In [102]:
all(person_trips['origin'][1:].values == person_trips['destination'][:-1].values)

True

In [108]:
tours.loc[5820]

person_id                           141
tour_type                          work
tour_type_count                       1
tour_type_num                         1
tour_num                              1
tour_count                            1
tour_category                 mandatory
number_of_participants                1
destination                         486
origin                              847
household_id                         41
start                                 6
end                                  13
duration                              7
tdd                                  26
composition                         NaN
tour_mode                   SHARED3FREE
atwork_subtour_frequency            eat
parent_tour_id                      NaN
stop_frequency                 1out_1in
primary_purpose                    work
Name: 5820, dtype: object

In [190]:
foo[1]

,Unnamed: 0,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode,inbound,tour_start,tour_end,topo_sorted
trip_id,,,,,,,,,,,,,,,,,
7465,2356341,22,5,933,school,1,True,1,school,886,821,7.0,SHARED3PAY,False,7.0,19.0,True
7470,2356343,22,5,933,school,2,False,3,othdiscr,821,314,15.0,SHARED2PAY,True,7.0,19.0,False
7469,2356342,22,5,933,school,1,False,3,othdiscr,314,886,18.0,SHARED2FREE,True,7.0,19.0,False
7471,2356344,22,5,933,school,3,False,3,Home,821,821,19.0,SHARED3FREE,True,7.0,19.0,False


In [195]:
for person_id, person_trips in tqdm(
        ordered_trips[ordered_trips['person_id'].isin(unsorted_persons)].groupby('person_id'), total=len(unsorted_persons)):
    for tour_id, tour_trips in person_trips.groupby('tour_id'):
        break
    break



  0%|          | 0/179981 [00:00<?, ?it/s]


In [197]:
tour_trips.sort_v

,Unnamed: 0,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode,inbound,tour_start,tour_end,topo_sorted
trip_id,,,,,,,,,,,,,,,,,
7465,2356341,22,5,933,school,1,True,1,school,886,821,7.0,SHARED3PAY,False,7.0,19.0,True
7470,2356343,22,5,933,school,2,False,3,othdiscr,821,314,15.0,SHARED2PAY,True,7.0,19.0,False
7469,2356342,22,5,933,school,1,False,3,othdiscr,314,886,18.0,SHARED2FREE,True,7.0,19.0,False
7471,2356344,22,5,933,school,3,False,3,Home,821,821,19.0,SHARED3FREE,True,7.0,19.0,False


In [77]:
trips['purpose'].value_counts()

Home        5123960
work        1765841
shopping    1469587
escort      1213669
othmaint     905923
school       813960
othdiscr     804964
eatout       679194
Work         438900
atwork       438900
social       390171
univ         149798
Name: purpose, dtype: int64

In [110]:
trips[trips['outbound'] == True]['purpose'].value_counts()

work        1637233
escort       886360
shopping     869975
school       813960
othdiscr     631296
othmaint     590434
atwork       438900
eatout       363197
social       248537
univ         147687
Name: purpose, dtype: int64

In [ ]:
outbound_purpose_order = {}

In [198]:
ordered_trips[ordered_trips['person_id'] == unsorted_persons[1]]

,Unnamed: 0,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode,inbound,tour_start,tour_end,topo_sorted
trip_id,,,,,,,,,,,,,,,,,
9497,2947242,28,6,1187,work,1,True,1,work,32,847,6.0,DRIVEALONEPAY,False,6.0,21.0,True
9502,2947244,28,6,1187,work,2,False,4,eatout,490,469,17.0,DRIVEALONEPAY,True,6.0,21.0,False
9501,2947243,28,6,1187,work,1,False,4,shopping,469,32,18.0,DRIVEALONEPAY,True,6.0,21.0,False
9503,2947245,28,6,1187,work,3,False,4,shopping,722,490,19.0,DRIVEALONEFREE,True,6.0,21.0,False
9504,2947246,28,6,1187,work,4,False,4,Home,847,722,21.0,DRIVEALONEPAY,True,6.0,21.0,True


In [119]:
ordered_trips[ordered_trips['person_id'] == 338]

,Unnamed: 0,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode,inbound,tour_start,tour_end
trip_id,,,,,,,,,,,,,,,,
111177,12964835,338,95,13897,work,1,True,1,work,70,821,7.0,DRIVEALONEFREE,False,7.0,16.0
110865,12964832,338,95,13858,atwork,1,True,2,escort,71,70,12.0,DRIVEALONEFREE,False,12.0,16.0
110866,12964833,338,95,13858,atwork,2,True,2,atwork,147,71,12.0,DRIVEALONEFREE,False,12.0,16.0
111181,12964836,338,95,13897,work,1,False,1,Home,821,70,16.0,DRIVEALONEPAY,True,7.0,16.0
110869,12964834,338,95,13858,atwork,1,False,1,Work,70,147,16.0,DRIVEALONEFREE,True,12.0,16.0


In [86]:
topo_sort_mask = (
            (tour_trips['destination'].shift() == tour_trips['origin']) |
            (tour_trips['person_id'].shift() != tour_trips['person_id']))

In [24]:
chrono_sort_mask = (
        (ordered_trips['depart'].shift() <= ordered_trips['depart']) |
        (ordered_trips['person_id'].shift() != ordered_trips['person_id']))

In [240]:
num_true, num_false = topo_sort_mask.value_counts().values

In [241]:
num_true

14108751

In [ ]:
num_fal

In [152]:
ordered_trips[ordered_trips['tour_id'] == 933]

,Unnamed: 0,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode,inbound,tour_start,tour_end,time_sort
trip_id,,,,,,,,,,,,,,,,,
7465,2356341,22,5,933,school,1,True,1,school,886,821,7.0,SHARED3PAY,False,7.0,19.0,7.0
7470,2356343,22,5,933,school,2,False,3,othdiscr,821,314,15.0,SHARED2PAY,True,7.0,19.0,19.0
7469,2356342,22,5,933,school,1,False,3,othdiscr,314,886,18.0,SHARED2FREE,True,7.0,19.0,19.0
7471,2356344,22,5,933,school,3,False,3,Home,821,821,19.0,SHARED3FREE,True,7.0,19.0,19.0


In [29]:
trips[trips['tour_id'] == 933][['person_id', 'tour_id', 'trip_num', 'destination', 'origin', 'depart', 'outbound']]

,person_id,tour_id,trip_num,destination,origin,depart,outbound
trip_id,,,,,,,
7465,22,933,1,886,821,7.0,True
7469,22,933,1,314,886,18.0,False
7470,22,933,2,821,314,15.0,False
7471,22,933,3,821,821,19.0,False


In [171]:
ordered_trips[ordered_trips['tour_id'] == 933][['person_id', 'tour_id', 'outbound', 'trip_num', 'purpose', 'destination', 'origin', 'depart', 'outbound']]

,person_id,tour_id,outbound,trip_num,purpose,destination,origin,depart,outbound
trip_id,,,,,,,,,
7465,22,933,True,1,school,886,821,7.0,True
7470,22,933,False,2,othdiscr,821,314,15.0,False
7469,22,933,False,1,othdiscr,314,886,18.0,False
7471,22,933,False,3,Home,821,821,19.0,False


#### joint tours never have atwork subtours

In [29]:
tours[tours['tour_category'] == 'joint']['atwork_subtour_frequency'].value_counts()

Series([], Name: atwork_subtour_frequency, dtype: int64)

#### joint tours whose members have atwork subtours?

In [37]:
atwork_tours = tours[(tours['tour_category'] == 'atwork')]

In [46]:
passengers_w_atwork_tours = jtp[(jtp['person_id'].isin(atwork_tours['person_id'])) & (jtp['participant_num'] > 1)]

#### types of joint tours whose passengers have their own atwork tours

In [51]:
tours[tours['tour_id'].isin(passengers_w_atwork_tours['tour_id'])]['tour_category'].value_counts()

joint    8530
Name: tour_category, dtype: int64

In [48]:
tours[tours['tour_id'].isin(passengers_w_atwork_tours['tour_id'])]['tour_type'].value_counts()

othdiscr    3498
eatout      1423
shopping    1391
othmaint    1352
social       866
Name: tour_type, dtype: int64

#### joint tours are never mandatory tours!?!

In [52]:
tours[tours['tour_category'] == 'joint']['tour_type'].value_counts()

othdiscr    35022
shopping    24667
othmaint    23753
eatout      13968
social      13133
Name: tour_type, dtype: int64

#### what do passenger's daily plans look like

In [55]:
plans[plans['person_id'].isin(passengers_w_atwork_tours.sample()['person_id'])]

,trip_id,person_id,PlanElementIndex,ActivityElement,ActivityType,x,y,departure_time
4424858,NaN,596338.0,1,activity,Home,-82.885191,42.585014,6.014
4424859,195599177.0,596338.0,2,leg,NaN,NaN,NaN,NaN
4424860,NaN,596338.0,3,activity,Home,-82.909918,42.579792,13.907
4424861,195598897.0,596338.0,4,leg,NaN,NaN,NaN,NaN
4424862,NaN,596338.0,5,activity,Home,-82.910130,42.440059,16.223
4424863,195598901.0,596338.0,6,leg,NaN,NaN,NaN,NaN
4424864,NaN,596338.0,7,activity,Home,-82.914809,42.420992,18.604
4424865,195599181.0,596338.0,8,leg,NaN,NaN,NaN,NaN
4424866,NaN,596338.0,9,activity,Home,-82.885191,42.585014,NaN


In [57]:
trips[trips['person_id'] == 596338].sort_values('depart')

,trip_id,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode
10692281,195599177,596338,229795,24449897,work,1,True,1,work,156,947,6.0,WALK_LOC
10692279,195598897,596338,229795,24449862,atwork,1,True,1,atwork,114,156,13.0,WALK
10692280,195598901,596338,229795,24449862,atwork,1,False,1,Work,156,114,16.0,WALK
10692282,195599181,596338,229795,24449897,work,1,False,1,Home,947,156,18.0,WALK_LOC


In [63]:
trips[trips['tour_id'].isin(tours[tours['tour_id'].isin(jtp[jtp['person_id'] == 596338]['tour_id'])]['tour_id'])]

,trip_id,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode
10692277,195598313,596335,229795,24449789,eatout,1,True,1,eatout,212,947,20.0,SHARED3FREE
10692278,195598317,596335,229795,24449789,eatout,1,False,1,Home,947,212,21.0,SHARED3PAY


# Run plan generation

In [5]:
from activitysim.abm.models.generate_beam_plans import get_trip_coords, generate_departure_times, sample_geoseries
from activitysim.abm.models.initialize_from_usim import get_zone_geoms_from_h3
import random

#### Create zones table

In [6]:
skims = pd.read_csv('https://beam-outputs.s3.amazonaws.com/output/detroit/detroit-200k__2020-05-29_22-09-51_mgd/ITERS/it.10/10.skimsOD.UrbanSim.Full.csv.gz')
h3_zone_ids = skims.origTaz.unique()

zone_geoms = get_zone_geoms_from_h3(h3_zone_ids)

zones = gpd.GeoDataFrame(h3_zone_ids, geometry=zone_geoms, crs="EPSG:4326")
zones.columns = ['h3_id', 'geometry']
zones['TAZ'] = list(range(1, len(h3_zone_ids) + 1))
zones = zones.set_index('TAZ')

block_geoms = gpd.read_file('../detroit/data/blocks.shp')
block_bounds = block_geoms.to_crs('EPSG:3078').unary_union

zones = zones.to_crs('EPSG:3078')
zones['geometry'] = zones['geometry'].intersection(block_bounds)
zones = zones.to_crs('EPSG:4326')

DriverError: ../detroit/data/blocks.shp: No such file or directory

#### Assign coords

In [ ]:
zones.geometry = zones.geometry.buffer(0)

In [ ]:
zones.reset_index(inplace=True)

In [ ]:
rand_point_zones = {}
for zone in zones[~zones['geometry'].is_empty].TAZ:
    size = 500
    polygon = zones[zones.TAZ == zone].geometry
    points = sample_geoseries(polygon, size, overestimate=2)
    rand_point_zones[zone] = points

In [ ]:
df = trips[['origin']].reset_index().drop_duplicates('trip_id')

In [ ]:
origins = []
for i, row in enumerate(df.itertuples(), 0):
    origins.append(random.choice(rand_point_zones[row.origin]))
origins = np.array(origins)

In [ ]:
df['origin_x'] = origins[:, 0]
df['origin_y'] = origins[:, 1]

In [46]:
df = df.set_index('trip_id').reindex(trips.index)
trips['origin_x'] = df['origin_x']
trips['origin_y'] = df['origin_y']

In [47]:
trips['origin_purpose'] = trips.groupby(
    'person_id')['purpose'].shift(periods=1).fillna('Home')
trips['x'] = trips.origin_x
trips['y'] = trips.origin_y

#### Assign departure times

In [35]:
df = trips[['person_id', 'depart']].reset_index().drop_duplicates('trip_id')
df['frac'] = np.random.rand(len(df),)

In [36]:
df.index.name = 'og_df_idx'

# PICK SHIT UP HERE ON MONDAY

Do we need to order by "tour_start"??? "trip_id"??? "inbound"???

In [ ]:
trips[trips['person_id'] == 650].sort_values(['person_id','depart','tour_start','inbound','trip_num'])

In [63]:
trips[trips['person_id'] == 650].sort_values(['person_id','tour_start','tour_id','inbound','trip_num'])

,Unnamed: 0,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode,origin_x,origin_y,origin_purpose,x,y,tour_start,inbound
trip_id,,,,,,,,,,,,,,,,,,,,
213233,10611254,650,187,26654,atwork,1,True,1,atwork,33,486,7.0,SHARED2FREE,-83.675583,42.666400,Home,-83.675583,42.666400,7.0,False
213513,10611257,650,187,26689,work,1,True,1,work,486,821,7.0,DRIVEALONEPAY,-83.657042,42.740672,Work,-83.657042,42.740672,7.0,False
213237,10611255,650,187,26654,atwork,1,False,2,othmaint,33,33,9.0,SHARED2FREE,-83.641132,42.621606,atwork,-83.641132,42.621606,7.0,True
213238,10611256,650,187,26654,atwork,2,False,2,Work,486,33,9.0,SHARED2PAY,-83.650398,42.582966,othmaint,-83.650398,42.582966,7.0,True
213517,10611258,650,187,26689,work,1,False,2,shopping,486,486,19.0,DRIVEALONEPAY,-83.703647,42.673458,work,-83.703647,42.673458,7.0,True
213518,10611259,650,187,26689,work,2,False,2,Home,821,486,19.0,DRIVEALONEFREE,-83.701100,42.666862,shopping,-83.701100,42.666862,7.0,True


In [62]:
trips[trips['person_id'] == 596338].sort_values(['person_id', 'depart', 'tour_start', 'inbound', 'trip_num'])

,Unnamed: 0,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode,origin_x,origin_y,origin_purpose,x,y,tour_start,inbound
trip_id,,,,,,,,,,,,,,,,,,,,
195599177,10692281,596338,229795,24449897,work,1,True,1,work,156,947,6.0,WALK_LOC,-82.893391,42.587223,Work,-82.893391,42.587223,6.0,False
195598897,10692279,596338,229795,24449862,atwork,1,True,1,atwork,114,156,13.0,WALK,-82.917582,42.421873,Home,-82.917582,42.421873,13.0,False
195598901,10692280,596338,229795,24449862,atwork,1,False,1,Work,156,114,16.0,WALK,-82.912157,42.437548,atwork,-82.912157,42.437548,13.0,True
195599181,10692282,596338,229795,24449897,work,1,False,1,Home,947,156,18.0,WALK_LOC,-82.920287,42.420552,work,-82.920287,42.420552,6.0,True


In [59]:
tours[tours.index.isin([26654, 26689])]

,Unnamed: 0,person_id,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination,...,start,end,duration,tdd,composition,tour_mode,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose
tour_id,,,,,,,,,,,,,,,,,,,,,
26689,4161579,650,work,1,1,1,1,mandatory,1,486.0,...,7.0,19.0,12.0,49.0,NaN,DRIVEALONEPAY,eat,NaN,0out_1in,work
26654,4374409,650,eat,1,1,1,1,atwork,1,33.0,...,7.0,9.0,2.0,39.0,NaN,SHARED2FREE,NaN,26689.0,0out_1in,atwork


In [37]:
trips.sort_values(['person_id','tour_start','tour_id','inbound','trip_num'])

In [23]:
ordered = df.sort_values(by=['person_id', 'depart', 'frac']).reset_index()

In [24]:
ordered[ordered['person_id'] == 650]

,og_df_idx,trip_id,person_id,depart,frac
1959,10611254,213233,650,7.0,0.266064
1960,10611257,213513,650,7.0,0.502631
1961,10611255,213237,650,9.0,0.391003
1962,10611256,213238,650,9.0,0.617725
1963,10611259,213518,650,19.0,0.145560
1964,10611258,213517,650,19.0,0.666612


In [ ]:
df2 = df.sort_values(by=['person_id', 'depart']).reset_index()
df2['fractional'] = ordered.frac

In [236]:
trips['departure_time'] = generate_departure_times(trips)

In [305]:
trips[trips['person_id'] == 650]

,trip_id,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,depart,trip_mode
10611254,213233,650,187,26654,atwork,1,True,1,atwork,33,486,7.0,SHARED2FREE
10611255,213237,650,187,26654,atwork,1,False,2,othmaint,33,33,9.0,SHARED2FREE
10611256,213238,650,187,26654,atwork,2,False,2,Work,486,33,9.0,SHARED2PAY
10611257,213513,650,187,26689,work,1,True,1,work,486,821,7.0,DRIVEALONEPAY
10611258,213517,650,187,26689,work,1,False,2,shopping,486,486,19.0,DRIVEALONEPAY
10611259,213518,650,187,26689,work,2,False,2,Home,821,486,19.0,DRIVEALONEFREE


In [238]:
cols = [
    'person_id', 'departure_time', 'purpose', 'origin',
    'destination', 'trip_mode', 'x', 'y']
trips = trips[cols].sort_values(
    ['person_id', 'departure_time']).reset_index()

In [252]:
trips.groupby('person_id')['destination'].last()[(trips.groupby('person_id')['destination'].last() != trips.groupby('person_id')['origin'].first())]

person_id
650        821
1080       821
1583       821
2604       907
2656       907
          ... 
4391267    138
4392020    138
4392190    138
4392310    857
4392983    174
Name: destination, Length: 14188, dtype: int64

In [ ]:
trips.groupby(['person_id', 'depart'])

In [254]:
trips[trips['person_id'] == 4391267]

,trip_id,person_id,departure_time,purpose,origin,destination,trip_mode,x,y
14136932,1440335577,4391267,8.234,atwork,991,324,DRIVEALONEPAY,-83.184186,42.194760
14136933,1440335581,4391267,8.498,Work,324,991,DRIVEALONEFREE,-83.171514,42.231517
14136934,1440335889,4391267,8.892,work,138,991,DRIVEALONEFREE,-83.266094,42.110917
14136935,1440335865,4391267,18.121,social,138,150,DRIVEALONEFREE,-83.313741,42.115840
14136936,1440335893,4391267,18.564,Home,991,138,DRIVEALONEFREE,-83.181533,42.204011
14136937,1440335869,4391267,21.062,shopping,150,836,DRIVEALONEFREE,-83.058149,42.340641
14136938,1440335870,4391267,21.541,Home,836,138,DRIVEALONEPAY,-83.053665,42.332312


In [205]:
return_trip = pd.DataFrame(
    trips.groupby('person_id').agg({'x': 'first', 'y': 'first'}),
    index=trips.person_id.unique())

In [206]:
trips = trips.append(return_trip)
trips.reset_index(inplace=True)
trips.person_id.fillna(trips['index'], inplace=True)

In [207]:
trips[trips['person_id'] == 596338]

,index,trip_id,person_id,departure_time,purpose,origin,destination,trip_mode,x,y,PlanElementIndex,ActivityType
1950908,1950908,195599177.0,596338.0,6.720,work,947.0,156.0,WALK_LOC,-82.893245,42.576168,1.0,Home
1950909,1950909,195598897.0,596338.0,13.748,atwork,156.0,114.0,WALK,-82.910921,42.423763,3.0,work
1950910,1950910,195598901.0,596338.0,16.302,Work,114.0,156.0,WALK,-82.902152,42.444910,5.0,atwork
1950911,1950911,195599181.0,596338.0,18.027,Home,156.0,947.0,WALK_LOC,-82.918037,42.422579,7.0,Work
14667037,596338,NaN,596338.0,NaN,NaN,NaN,NaN,NaN,-82.893245,42.576168,NaN,NaN


In [208]:
trips['PlanElementIndex'] = trips.groupby('person_id').cumcount() * 2 + 1
trips = trips.sort_values(
    ['person_id', 'departure_time']).reset_index(drop=True)

In [209]:
trips[trips['person_id'] == 596338]

,index,trip_id,person_id,departure_time,purpose,origin,destination,trip_mode,x,y,PlanElementIndex,ActivityType
2473950,1950908,195599177.0,596338.0,6.720,work,947.0,156.0,WALK_LOC,-82.893245,42.576168,1,Home
2473951,1950909,195598897.0,596338.0,13.748,atwork,156.0,114.0,WALK,-82.910921,42.423763,3,work
2473952,1950910,195598901.0,596338.0,16.302,Work,114.0,156.0,WALK,-82.902152,42.444910,5,atwork
2473953,1950911,195599181.0,596338.0,18.027,Home,156.0,947.0,WALK_LOC,-82.918037,42.422579,7,Work
2473954,596338,NaN,596338.0,NaN,NaN,NaN,NaN,NaN,-82.893245,42.576168,9,NaN


In [210]:
trips['ActivityType'] = trips.groupby('person_id')['purpose'].shift(periods=1).fillna('Home')

In [216]:
trips[trips['person_id'] == 596338]

,index,trip_id,person_id,departure_time,purpose,origin,destination,trip_mode,x,y,PlanElementIndex,ActivityType,ActivityElement
2473950,1950908,195599177.0,596338.0,6.720,work,947.0,156.0,WALK_LOC,-82.893245,42.576168,1,Home,activity
2473951,1950909,195598897.0,596338.0,13.748,atwork,156.0,114.0,WALK,-82.910921,42.423763,3,work,activity
2473952,1950910,195598901.0,596338.0,16.302,Work,114.0,156.0,WALK,-82.902152,42.444910,5,atwork,activity
2473953,1950911,195599181.0,596338.0,18.027,Home,156.0,947.0,WALK_LOC,-82.918037,42.422579,7,Work,activity
2473954,596338,NaN,596338.0,NaN,NaN,NaN,NaN,NaN,-82.893245,42.576168,9,Home,activity


In [212]:
trips['ActivityElement'] = 'activity'

In [224]:
legs = trips[~pd.isnull(trips['trip_id'])]
legs['PlanElementIndex'] = legs['PlanElementIndex'] + 1
legs['ActivityElement'] = 'leg'
legs['ActivityType'] = None
trips['trip_id'] = None

In [225]:
legs[legs['person_id'] == 596338]

,index,trip_id,person_id,departure_time,purpose,origin,destination,trip_mode,x,y,PlanElementIndex,ActivityType,ActivityElement
2473950,1950908,195599177.0,596338.0,6.720,work,947.0,156.0,WALK_LOC,-82.893245,42.576168,2,Home,leg
2473951,1950909,195598897.0,596338.0,13.748,atwork,156.0,114.0,WALK,-82.910921,42.423763,4,work,leg
2473952,1950910,195598901.0,596338.0,16.302,Work,114.0,156.0,WALK,-82.902152,42.444910,6,atwork,leg
2473953,1950911,195599181.0,596338.0,18.027,Home,156.0,947.0,WALK_LOC,-82.918037,42.422579,8,Work,leg


In [226]:
trips = trips.append(legs).sort_values(['person_id', 'PlanElementIndex'])

In [227]:
trips[trips['person_id'] == 596338]

,index,trip_id,person_id,departure_time,purpose,origin,destination,trip_mode,x,y,PlanElementIndex,ActivityType,ActivityElement
2473950,1950908,195599177.0,596338.0,6.720,work,947.0,156.0,WALK_LOC,-82.893245,42.576168,1,Home,activity
2473950,1950908,195599177.0,596338.0,6.720,work,947.0,156.0,WALK_LOC,-82.893245,42.576168,2,Home,leg
2473951,1950909,195598897.0,596338.0,13.748,atwork,156.0,114.0,WALK,-82.910921,42.423763,3,work,activity
2473951,1950909,195598897.0,596338.0,13.748,atwork,156.0,114.0,WALK,-82.910921,42.423763,4,work,leg
2473952,1950910,195598901.0,596338.0,16.302,Work,114.0,156.0,WALK,-82.902152,42.444910,5,atwork,activity
2473952,1950910,195598901.0,596338.0,16.302,Work,114.0,156.0,WALK,-82.902152,42.444910,6,atwork,leg
2473953,1950911,195599181.0,596338.0,18.027,Home,156.0,947.0,WALK_LOC,-82.918037,42.422579,7,Work,activity
2473953,1950911,195599181.0,596338.0,18.027,Home,156.0,947.0,WALK_LOC,-82.918037,42.422579,8,Work,leg
2473954,596338,NaN,596338.0,NaN,NaN,NaN,NaN,NaN,-82.893245,42.576168,9,Home,activity


In [ ]:
trips[trips['Activity']]